In [6]:
from __future__ import division
from __future__ import print_function

## Breakpoints and Strips

In [7]:
def hasBreakpoints(seq):
    """ returns True if sequences is not strictly increasing by 1 """
    for i in range(1, len(seq)):
        if (seq[i] != seq[i-1] + 1):
            return True
    return False

def getStrips(seq):
    """ find contained intervals where sequence is ordered, and return intervals
    in as lists, increasing and decreasing. Single elements are considered
    decreasing. "Contained" excludes the first and last interval. """
    deltas = [seq[i+1] - seq[i] for i in range(len(seq)-1)]
    increasing = list()
    decreasing = list()
    start = 0
    for i, diff in enumerate(deltas):
        if (abs(diff) == 1) and (diff == deltas[start]):
            continue
        if (start > 0):
            if deltas[start] == 1:
                increasing.append((start, i+1))
            else:
                decreasing.append((start, i+1))
        start = i+1
    return increasing, decreasing

In [8]:
pi = [0,1,2,3,4,5,6,7,8,9]
print(hasBreakpoints(pi))
increasing, decreasing = getStrips(pi)

for start, end in increasing:
    print("increasing: (%d, %d): %s" % (start, end, str(pi[start:end])))
for start, end in decreasing:
    print("decreasing: (%d, %d): %s" % (start, end, str(pi[start:end])))
bpoints = len(increasing) + len(decreasing)
if (bpoints > 0):
    bpoints += 1
print(bpoints)

False
0


## Handle Reversals

In [9]:
def pickReversal(seq, strips):
    """ test each decreasing interval to see if it leads to a reversal that
    removes two breakpoints, otherwise, return a reversal that removes only one """
    for i, j in strips:
        k = seq.index(seq[j-1]-1)
        if (seq[k+1] + 1 == seq[j]):
            # removes 2 breakpoints
            return 2, (min(k+1, j), max(k+1, j))
    # In the worst case we remove only one, but avoid the length "1" strips
    for i, j in strips:
        k = seq.index(seq[j-1]-1)
        if (j - i > 1):
            break
    return 1, (min(k+1, j), max(k+1, j))

def doReversal(seq,reversal):
    i, j = reversal
    return seq[:i] + [element for element in reversed(seq[i:j])] + seq[j:]

## Let's Do it

In [15]:
def improvedBreakpointReversalSort(seq, verbose=True):
    seq = [0] + seq + [max(seq)+1]                             # Extend sequence
    if (verbose):
        print("0: %s" % seq)
    N = 0
    while hasBreakpoints(seq):
        increasing, decreasing = getStrips(seq)
        if len(decreasing) > 0:                                # pick a reversal that removes a decreasing strip
            removed, reversal = pickReversal(seq, decreasing)
        else:
            removed, reversal = 0, increasing[0]               # No breakpoints can be removed
        if verbose:
            print("Strips:", increasing, decreasing)
            print("%d: %s  rho%s" % (removed, seq, reversal))
        seq = doReversal(seq,reversal)
        N += 1
    if verbose:
        print(seq, "Sorted")
    return N

# Also try: [1,9,3,4,7,8,2,6,5]
print(improvedBreakpointReversalSort([3,4,1,2,5,6,7,10,9,8], verbose=True))

0: [0, 3, 4, 1, 2, 5, 6, 7, 10, 9, 8, 11]
Strips: [(1, 3), (3, 5), (5, 8)] [(8, 11)]
2: [0, 3, 4, 1, 2, 5, 6, 7, 10, 9, 8, 11]  rho(8, 11)
Strips: [(1, 3), (3, 5)] []
0: [0, 3, 4, 1, 2, 5, 6, 7, 8, 9, 10, 11]  rho(1, 3)
Strips: [(3, 5)] [(1, 3)]
1: [0, 4, 3, 1, 2, 5, 6, 7, 8, 9, 10, 11]  rho(3, 5)
Strips: [] [(1, 5)]
2: [0, 4, 3, 2, 1, 5, 6, 7, 8, 9, 10, 11]  rho(1, 5)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] Sorted
4
